# Associated Currents

Noteboook to train and test machine learning models to associated currents in the frequency domain.

#### Machine Learning Models:
    - Nearest Centroid
    - K - Nearest Neighbors Classifier
    - Gaussian Naive Bayes
    - Multilayer Perceptron

## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score

## Dataset

In [58]:
dataset = pd.read_csv('../CreatingDataset/FrequencyDomain/Dataset.csv')
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,3.93,2.513044,8.163932,33.629912,7.659697,6.725006,10.643506,13.177546,59.482317,10.749250,...,1.888285,2.273819,1.312697,1.323685,2.193599,2.599519,2.390184,3.165756,0.534784,Notebook
1,1.00,0.191735,4.678745,23.926314,8.975643,6.839434,8.616491,10.477577,54.968947,3.580283,...,0.184238,1.160567,0.852239,1.300667,0.425774,1.524319,0.897228,1.132210,0.852345,Notebook
2,3.07,3.250578,3.296642,28.901722,7.726107,9.185737,9.997531,13.401718,53.479286,8.655162,...,0.802962,0.547124,0.942776,0.718461,0.779437,0.926345,1.893492,0.777703,0.770305,Notebook
3,1.94,0.968233,4.019366,27.971932,10.626979,7.588198,9.675171,14.644327,55.259634,3.368294,...,1.568216,1.422735,0.251880,0.763626,1.406309,1.429444,0.035365,1.100480,0.615161,Notebook
4,4.44,1.646007,4.716094,32.419320,8.428120,8.383729,11.016293,12.865954,56.504946,12.448456,...,2.359542,0.006621,0.500235,0.081978,0.039892,2.824678,1.531935,0.927873,1.221977,Notebook


## Partição Holdout 80/20

In [59]:
device_position = '100'

y = dataset[device_position]
x = dataset.drop([device_position], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20)

## Treinamento e Teste Centróides mais próximas

In [60]:
nc = NearestCentroid()
nc.fit(X_train, Y_train)
previsaoNC = nc.predict(X_test)

print('Acurácia:',accuracy_score(Y_test, previsaoNC))

Acurácia: 0.49666666666666665


## Treinamento e Teste KNN; K=5

In [61]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, Y_train)

previsaokNN = knn.predict(X_test)

print('Acurácia:',accuracy_score(Y_test, previsaokNN))

Acurácia: 0.5133333333333333


## Treino e Teste Gaussian Naive Bayes

In [62]:
gnb = GaussianNB()
gnb.fit(X_train, Y_train)

previsaoGNB = gnb.predict(X_test)

accuracy_score(Y_test, previsaoGNB)

0.7133333333333334

## Treinamento e Teste MLP

In [81]:
MLP = MLPClassifier(hidden_layer_sizes=(30,25,20),
                     max_iter=300, 
                     activation='identity', 
                     learning_rate='constant',
                     learning_rate_init=0.005,
                     batch_size = 3,
                     solver='lbfgs')
MLP.fit(X_train, Y_train)
previsaoMLP = MLP.predict(X_test)

print('Acurácia:', accuracy_score(Y_test, previsaoMLP))

Acurácia: 0.8233333333333334
